# What kind of plots

In [1]:
import re
import numpy as np
from mapper import D
from tqdm.auto import tqdm

### ToDo
- Need to calculate Todo for engineering fields
- Metric to compare distributions

Plots
- Distribution difference between y19 and y20, kde plots (eng, noneng) (cgpa/percent?)
- Distribution difference between y19 and y20, kde plots (individual) (cgpa/percent?)
- Same thing but by gender
- Histograms of above 9 pointers (eng, noneng)
- Histograms of above 9 pointers (eng)
- Some gender related pichart
- which subj is difficult (comps)
- Change in plots by subj
- Differences by center (top colleges?)
- Differences for my college

In [2]:
d = D()

In [3]:
years = ("y19", "y20")
non_eng = ('bcom_fm', 'bed', 'bcom_af', 'mms', 'llb', 'bpharm', 'ba', 'bcom_bi')
eng = ('comp', 'chem', 'electronics', 'civil', 'electrical', 'mech', 'it', 'extc')

In [26]:
def check_columns(columns, d=d):
    for col in columns:
        print(f"checking for : {col}")
        for br in non_eng + eng:
            for y in years:
                temp = getattr(d, y)
                temp = getattr(temp, br)
                if not col in temp.df.columns:
                    print(f"{y} : {br}")
        print()

In [28]:
check_columns(['total_marks', 'gpa'])

checking for : total_marks
y19 : bed
y20 : bed
y19 : mms
y20 : mms
y19 : bpharm
y20 : bpharm
y19 : comp
y19 : chem
y19 : electronics
y19 : civil
y19 : electrical
y19 : mech
y19 : it
y19 : extc

checking for : gpa
y19 : llb
y20 : llb



- somethigns messed in the calculations
- need to use subj breakdown to cross verify the total from columns
- need to assert that totals are below max for y20

In [231]:
df = d.y19.comp.df.copy()

In [290]:
cdf = fix_and_aug_df(d.y19.comp)
nc, mc, mt = get_cols_and_tot(d.y19.com)

In [383]:
temp = get_fixed_and_auged_df(d.y20.comp)

In [20]:
non_eng

('bcom_fm', 'bed', 'bcom_af', 'mms', 'llb', 'bpharm', 'ba', 'bcom_bi')

In [25]:
d.y19.ba.df.columns

Index(['Unnamed: 0', 'no', 'name', 'is_female', 'center', 'total_marks', 'ac',
       'acg', 'gpa'],
      dtype='object')

In [28]:
d.y19.ba.df.columns.tolist()

['Unnamed: 0',
 'no',
 'name',
 'is_female',
 'center',
 'total_marks',
 'ac',
 'acg',
 'gpa']

For the ones where total_marks is present find the max_total_marks, for everything else use gpa, only llb doesn't have gpa

In [33]:
MAX_TOTAL_MARKS_DICT = {
    'bcom_fm': 600,
    'bcom_af': 600,
    'bcom_bi': 600,
    'llb': 400,
    'ba': 600,
}

In [32]:
non_eng

('bcom_fm', 'bed', 'bcom_af', 'mms', 'llb', 'bpharm', 'ba', 'bcom_bi')

In [31]:
for b in non_eng:
    print(b)
    for y in years:
        temp = getattr(d, y)
        temp = getattr(temp, b)
        print(y, ', '.join(temp.df.columns.tolist()))
    print()


bcom_fm
y19 Unnamed: 0, no, name, is_female, center, total_marks, ac, acg, gpa
y20 Unnamed: 0, no, name, is_female, center, total_marks, ac, acg, gpa

bed
y19 Unnamed: 0, no, name, is_female, center, ac, acg, gpa
y20 Unnamed: 0, no, name, is_female, center, ac, acg, gpa

bcom_af
y19 Unnamed: 0, no, name, is_female, center, total_marks, ac, acg, gpa
y20 Unnamed: 0, no, name, is_female, center, total_marks, ac, acg, gpa

mms
y19 Unnamed: 0, no, name, is_female, center, ac, acg, gpa
y20 Unnamed: 0, no, name, is_female, center, ac, acg, gpa

llb
y19 Unnamed: 0, no, name, is_female, center, 1_100, 2_100, 3_100, 4_100, total_marks
y20 Unnamed: 0, no, name, is_female, center, 1_100, 2_100, 3_100, 4_100, total_marks

bpharm
y19 Unnamed: 0, no, name, is_female, center, ac, acg, gpa
y20 Unnamed: 0, no, name, is_female, center, ac, acg, gpa

ba
y19 Unnamed: 0, no, name, is_female, center, total_marks, ac, acg, gpa
y20 Unnamed: 0, no, name, is_female, center, total_marks, ac, acg, gpa

bcom_bi
y19

In [7]:
temps = []
for b in tqdm(eng, leave=False):
    for y in years:
        temp = getattr(d, y)
        temp = getattr(temp, b)
        temps.append(get_fixed_and_auged_df(temp))

In [5]:
def get_fixed_and_auged_df(dcourse):
    """
    Function that fixes the numeric values and
    maybe adds a total and percentage column.
    """
    df = dcourse.df.copy()
    
    num_cols, mark_cols, max_tot = get_cols_and_tot(dcourse)
    df = clean_numerics(df, num_cols)
    if 'total_marks' in df.columns:
        df = fix_tot_col(df)
        
    df = df.fillna(0)
    
    if 'total_marks' not in df.columns:
        df = append_tot_col(df, mark_cols)
        
    df = append_percent_col(df, max_tot)
    return df

def get_cols_and_tot(dcourse):
    if hasattr(dcourse, 'subj_break_down'):
        sb = dcourse.subj_break_down
        return (
            get_all_num_cols_from_sb(sb),
            *get_mark_cols_and_max_tot_from_sb(sb)
        )
    else:
        raise NotImplementedError("yo wtf?!")

def clean_numerics(df, num_cols):
    df = df.apply(fix_numeric_values, axis=1, args=(num_cols, ))
    return df

def append_tot_col(df, marks_cols):
    marks = df[marks_cols]
    df['total_marks'] = marks.sum(axis=1)
    return df

def append_percent_col(df, max_tot):
    df['percent'] = df['total_marks'] / max_tot
    return df

def fix_tot_col(df):
    return df.apply(get_fixed_tm, axis=1)

def get_fixed_tm(row):
    tm = row['total_marks']
    if "NA" in tm:
        tm = np.nan
    elif "/" in tm:
        tm = tm.split("/")[0]
        tm = tm.split("@")[0]
        tm = float(tm)
    else:
        tm = float(tm)
    row['total_marks'] = tm
    return row

def fix_numeric_values(row, num_cols):
    for c in num_cols:
        row[c] = convert_numeric(row[c])
    return row

ex_num = lambda v: re.findall("(\d+(?:\.\d+)?)\w?", v)
def convert_numeric(v):
    if v in ['--', 'A']:
        return np.nan
    elif isinstance(v, str):
        return float(ex_num(v)[0])
    else:
        return float(v)

In [6]:
def get_mark_tups_from_sb(sb):
    return [(k,v) for k, i in sb.items() for v in i[0] if re.findall("^\d+/\d+$", v)]

def get_all_num_cols_from_sb(sb):
    return [f"{k}_{v}" for k, i in sb.items() for v in i[0] if not re.findall("^G$", v)] + ['ac', 'acg', 'gpa']

def get_mark_cols_from_mt(mt):
    return [f"{k}_{v}" for k,v in mt]

def get_max_tot_from_mt(mt):
    return sum([int(v.split("/")[0]) for _, v in mt])

def get_mark_cols_and_max_tot_from_sb(sb):
    # From subject break down
    mt = get_mark_tups_from_sb(sb)
    mc = get_mark_cols_from_mt(mt)
    mx = get_max_tot_from_mt(mt)
    return mc, mx

In [229]:
num_cols = get_all_num_cols_from_sb(sb)
ex_num = lambda v: re.findall("(\d+(?:\.\d+)?)\w?", v)

for i, row in d.y19.comp.df.iterrows():
    to_break = False
    for c in num_cols:
        try:
            v = row[c]
            if isinstance(v, str):
                v = ex_num(row[c])[0]
            float(v)
        except:
            if row[c] in ['--', 'A']: continue
            print(i, c, row[c], row)
            to_break = True
            break
    if to_break: break
        
#     break

numbers that can't be converted:
- '--' 
- 'A' Absent
- wrong stuff?

data
- pie of 9 and up
- for atleast one which subj is diff

Notes
- section 1, data
- open with goodharts law paraphrase
- section 2, thoughts
    - personal experience
- open with killer mike
- cat's cradle bokononism